In [70]:
from pvml import naivebayes as nb
import numpy
from collections import Counter as counter
import glob
import re

In [96]:
def read_file(flist):
    k=[]
    for f in flist:
        f=open(f, "r")
        for line in f:
            #for word in re.split('; |, |\s|\n|<br|/>|--',line): #fancy splitting 
            for word in line.split():
                k.append(word)
    return k

def file_list(path, pn): #pn stands for positive/negative
    files = [f for f in glob.glob(path + "/" + pn + "**/*.txt", recursive=True)]
    return files

def build_dict(path, n, save):
    fl=file_list(path, "")
    words=read_file(fl)
    dic=counter(words).most_common(n)
    if save==False:
        return dic
    else:
        f=open("dictionary.txt", "w")
        for w in [word for word, word_count in dic]:
            f.write(str(w)+"\n")
        f.close()
        


path="aclImdb/smalltrain"
n=1000

None
